****
# Import transformation(30102023)

In [35]:
import pandas as pd
import numpy as np

%run Mylib_import.ipynb
exp_dir = "Arisa_NNSIFT"

# Specify the path to your Excel file
folder_name = 'cap_1' #'coffee_mug_1'
experiment_fol = 'test30102023'
excel_file_path = f'{exp_dir}/{experiment_fol}/Wahington-Lai_results_{folder_name}.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)
all_T_arrays = []
all_src_name =[]
all_tar_name=[]
# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Get the transformation string
    transformation_str = row['Transformation']
    
    # Split the string into lines
    lines = transformation_str.splitlines()
    
    # Process each line
    transformation_array = []
    for line in lines:
        # Remove brackets and extra spaces
        line = line.replace("[", "").replace("]", "").strip()
        
        # Skip empty lines
        if line:
            # Split the line into elements
            elements = line.split()
            
            # Convert each element to a float if it's not 'None'
            float_elements = []
            for e in elements:
                try:
                    float_elements.append(float(e))
                except ValueError:
                    float_elements.append(None)  # or you can skip this element entirely
            
            transformation_array.append(float_elements)
            
    all_src_name.append(row['Source File'])
    all_tar_name.append(row['Target File'])
    all_T_arrays.append(transformation_array)
    # Convert the list of lists to a numpy array
    transformation_array = np.array(transformation_array)
    
#     # Save the numpy array to a text file
#     np.savetxt(f"{exp_dir}/{experiment_fol}/transformation/{folder_name}/{row['Source File']}_{row['Target File']}.txt", transformation_array, fmt="%s")
all_T_arrays

[[[0.999976184, -0.0068177472, -0.00107271818, -0.000171268152],
  [0.00682181676, 0.99996937, 0.00383690398, -1.22154835e-05],
  [0.00104652628, -0.00384413049, 0.999992064, 0.000325784305],
  [0.0, 0.0, 0.0, 1.0]],
 [[0.99546071, 0.04690079, -0.08281478, -0.05664111],
  [-0.04830415, 0.99871969, -0.01502315, -0.01257069],
  [0.08200415, 0.01895525, 0.99645171, -0.00180845],
  [0.0, 0.0, 0.0, 1.0]],
 [[0.99859078, 0.0178107, -0.04999225, -0.03287949],
  [-0.01954305, 0.99921773, -0.03438028, -0.02641217],
  [0.0493408, 0.03530883, 0.99815769, -0.00189214],
  [0.0, 0.0, 0.0, 1.0]],
 [[0.996809473, 0.0797828557, 0.00236012449, -0.00247321182],
  [-0.0797815871, 0.996812171, -0.000627044642, -0.000586321359],
  [-0.00240262823, 0.000436749562, 0.999997018, 0.00306175201],
  [0.0, 0.0, 0.0, 1.0]],
 [[0.89363304, 0.43968042, -0.0900062, -0.0315735],
  [0.38921023, -0.85909413, -0.3323743, -0.24218503],
  [-0.22346227, 0.26198932, -0.93884302, -1.3560396],
  [0.0, 0.0, 0.0, 1.0]],
 [[0.9903

****
# Check results(31102023)

In [5]:
loop_n = 139
source_dir_test = f"D:/Users/msari/datasets/Wahington-Lai-rgbd/{folder_name[:-2]}/{folder_name}/{all_src_name[loop_n]}.ply"
target_dir_test = f"D:/Users/msari/datasets/Wahington-Lai-rgbd/{folder_name[:-2]}/{folder_name}/{all_tar_name[loop_n]}.ply"
print(f"source: {source_dir_test}, \ntarget: {target_dir_test}")
src_pcd = o3d.io.read_point_cloud(source_dir_test)
tar_pcd = o3d.io.read_point_cloud(target_dir_test)
all_T_arrays[loop_n]

source: D:/Users/msari/datasets/Wahington-Lai-rgbd/cap/cap_1/cap_1_1_151.ply, 
target: D:/Users/msari/datasets/Wahington-Lai-rgbd/cap/cap_1/cap_1_1_156.ply


[[1.0, 0.0, 0.0, 0.00315425],
 [0.0, 1.0, 0.0, 0.00671351],
 [0.0, 0.0, 1.0, -0.00700003],
 [0.0, 0.0, 0.0, 1.0]]

In [6]:
src_ = copy.deepcopy(src_pcd)
tar_ = copy.deepcopy(tar_pcd)
transrc_= copy.deepcopy(src_pcd)
o3d.visualization.draw_geometries([src_, tar_], width=800, height=600)
transrc_.transform(all_T_arrays[loop_n])
o3d.visualization.draw_geometries([transrc_, tar_], width=800, height=600)

In [28]:
%run Mylib_import.ipynb
exp_dir = "Arisa_NNSIFT"
create_folder(exp_dir)

def process_point_clouds_dim(src, tar, imdim):   
    check_cl_src = check_pc_has_color(src)
    check_cl_tar = check_pc_has_color(tar)

    def compute_transformation_using_fpfh(src, tar):
        # Compute FPFH features
        source_fpfh = compute_fpfh_features(src)
        target_fpfh = compute_fpfh_features(tar)

        # Match the features
        matches = o3d.registration.registration_ransac_based_on_feature_matching(
            src, tar, source_fpfh, target_fpfh,
            max_correspondence_distance=0.1,
            estimation_method=o3d.registration.TransformationEstimationPointToPoint(False),
            ransac_n=4,
            checkers=[o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                      o3d.registration.CorrespondenceCheckerBasedOnDistance(0.1)],
            criteria=o3d.registration.RANSACConvergenceCriteria(4000000, 500)
        )

        return matches.transformation, 0

    if check_cl_src == 'No' or check_cl_tar == 'No':
        print("----- Non-colored point clouds ------")
        transformation, if_col = compute_transformation_using_fpfh()
        imdim = 0
        num_keypoints = [0, 0]
        
    else:
        print("----- Colored point clouds ------")
        # calculates img_dim referring pointclouds sizes
        #imdim = 50
        
        # Generate pseudo-images (color maps) for both point clouds:    
        src_colormap, src_pointmap = point_cloud_to_color_and_point_map(src, img_dim=imdim)
        tar_colormap, tar_pointmap  = point_cloud_to_color_and_point_map(tar, img_dim=imdim)

        # Extracting SIFT keypoints and descriptors: from the pseudo-images.   
        src_keypoints, src_descriptors = extract_sift_feat(src_colormap)
        tar_keypoints, tar_descriptors = extract_sift_feat(tar_colormap)
        print(f"Number of source keypoints extracted: {len(src_keypoints), len(tar_keypoints)}")
        draw_keypoints(src_colormap, src_keypoints, 'Source pseudo-image Keypoints')
        draw_keypoints(tar_colormap, tar_keypoints, 'Target pseudo-image Keypoints')
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        if len(src_keypoints) <= 2 or len(tar_keypoints) <= 2:
            print("----- Keypoints are not proper detected ------")
            transformation, if_col = compute_transformation_using_fpfh()
            num_keypoints = [len(src_keypoints), len(tar_keypoints)]
        else:
            if_col = 1
            # Project the keypoints from 2D to 3D using the point map
            src_keypoints_3d, src_pseudesc = project_keypoints_to_3D(src_keypoints, src_descriptors, src_pointmap)
            tar_keypoints_3d, tar_pseudesc = project_keypoints_to_3D(tar_keypoints, tar_descriptors, tar_pointmap)

            # Debugging: Printing the number of 3D keypoints generated
            print(f"Number of 3D keypoints(src, tar): {len(src_keypoints_3d), len(tar_keypoints_3d)}")
            num_keypoints = [len(src_keypoints_3d), len(tar_keypoints_3d)]

            # Assuming src_descriptors and tar_descriptors are your source and target descriptors
            good_matches = find_good_matches(src_pseudesc, tar_pseudesc)

            # Assuming good_matches contains your filtered matches
            matched_src_keypoints, matched_tar_keypoints, matched_src_descriptors, matched_tar_descriptors = \
                extract_matched_keypoints_descriptors(good_matches, src_keypoints_3d, tar_keypoints_3d, src_pseudesc, tar_pseudesc)

            # Applying RANSAC
            transformation = apply_ransac(matched_src_keypoints, matched_tar_keypoints, matched_src_descriptors, matched_tar_descriptors)
    
    # Check if the transformation was successful
    if transformation is not None:
        transformed_src = copy.deepcopy(src)
        transformed_src.transform(transformation)
    else:
        print("Transformation failed.")
        print(transformation)
        return None, None, None, None, None, imdim, num_keypoints, if_col

    # Compute and print RMSE for quantitative evaluation
    init_rmse = compute_rmse(src, tar)
    rmse = compute_rmse(transformed_src, tar)

    # Compute color differences
    mean_color_init,mean_point_init  = compute_differences(src, tar)
    mean_color_regist, mean_point_regist = compute_differences(transformed_src, tar)
    
    return transformed_src, rmse, mean_color_regist, mean_point_regist, transformation, imdim, num_keypoints, if_col

###############################
dim = 40
transformed_src, rmse, mean_color_regist, mean_point_regist, transformation, imdim, num_keypoints, if_col = process_point_clouds_dim(src_, tar_, dim)

src_ = copy.deepcopy(src_pcd)
tar_ = copy.deepcopy(tar_pcd)
print(f"source:{len(src_.points)} points, target:{len(tar_.points)} points")
transrc_= copy.deepcopy(src_pcd)
o3d.visualization.draw_geometries([src_, tar_], width=800, height=600)
transrc_.transform(transformation)
o3d.visualization.draw_geometries([transrc_, tar_], width=800, height=600)
o3d.visualization.draw_geometries([transformed_src, tar_], width=800, height=600)

Folder 'Arisa_NNSIFT' already exists.


In [34]:
def compute_transformation_using_fpfh(src, tar):
    # Compute FPFH features
    source_fpfh = compute_fpfh_features(src)
    target_fpfh = compute_fpfh_features(tar)

    # Match the features
    matches = o3d.registration.registration_ransac_based_on_feature_matching(
        src, tar, source_fpfh, target_fpfh,
        max_correspondence_distance=0.1,
        estimation_method=o3d.registration.TransformationEstimationPointToPoint(False),
        ransac_n=4,
        checkers=[o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                  o3d.registration.CorrespondenceCheckerBasedOnDistance(0.1)],
        criteria=o3d.registration.RANSACConvergenceCriteria(4000000, 500)
    )

    return matches.transformation, 0
T,_ = compute_transformation_using_fpfh(src_, tar_)

src_ = copy.deepcopy(src_pcd)
tar_ = copy.deepcopy(tar_pcd)
print(f"source:{len(src_.points)} points, target:{len(tar_.points)} points")
transrc_= copy.deepcopy(src_pcd)
o3d.visualization.draw_geometries([src_, tar_], width=800, height=600)
transrc_.transform(T)
o3d.visualization.draw_geometries([transrc_, tar_], width=800, height=600)
T

source:16681 points, target:16331 points


array([[ 9.82705233e-01,  1.84871946e-01,  1.06202166e-02,
         1.05112037e-02],
       [-1.84178382e-01,  9.81745695e-01, -4.74732895e-02,
        -3.55883946e-02],
       [-1.92028313e-02,  4.46962357e-02,  9.98816048e-01,
        -3.79827136e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])